## Code to build the user graph dataframe

In [1]:
import numpy as np
import pandas as pd
import sqlite3
from os import getcwd

In [62]:
# Download a filtered (~900) set of (user/track/num_plays) triples from SQLite
con = sqlite3.connect('track_metadata.db')
reduced_triples = pd.read_sql_query("SELECT * FROM balanced_subset", con)
con.close()

In [67]:
# Download song metadata information (artist_name, album, release year) for the filtered tracks
con = sqlite3.connect('track_metadata.db')
song_info = pd.read_sql_query("SELECT * FROM subset_songs", con)
con.close()
song_info.set_index(['track_id'],inplace=True) # make track_id row key

In [63]:
# Initialize a dataframe, indexed by user_id
user_df = pd.DataFrame()
# sort user_id(s) in increasing order, this makes groupby an easy way to correctly join in new columns
user_df['user_id'] = np.sort(np.unique(reduced_triples['user_id']))
user_df.set_index(['user_id'],inplace=True)

# Add columns for number of plays and duration of plays
user_df['num_listens'] = reduced_triples.groupby('user_id')['num_plays'].sum()
user_df['listening_duration'] = reduced_triples.groupby('user_id')['duration'].sum() # in minutes
user_df.listening_duration = user_df.listening_duration.astype('int32')

In [69]:
# For each user build two dictionaries which will become columns
user_to_tracks = reduced_triples.groupby('user_id')['track_id'].apply(list).to_dict()
user_track_list = [] # Will hold dictionaries for each user and be turned into a columns
user_artist_list = []
for user_id, songs in user_to_tracks.items():
    track_dict = {} # maps track_id to num_plays
    artist_dict = {} # maps artist_id to num_plays
    for song in songs:
        num_plays = reduced_triples.loc[(reduced_triples['user_id']==user_id) & (reduced_triples['track_id']==song)]['num_plays']
        track_dict[song] = int(num_plays)
        artist_id = song_info.loc[song]['artist_id']
        if artist_id not in artist_dict: artist_dict[artist_id] = int(num_plays)
        else: artist_dict[artist_id] = artist_dict[artist_id] + int(num_plays)
        
    user_track_list.append(track_dict)
    user_artist_list.append(artist_dict)
    
# Create new columns
user_df['song_map'] = pd.Series(user_track_list,index=user_df.index.tolist())
user_df['artist_map'] = pd.Series(user_artist_list,index=user_df.index.tolist())

In [84]:
#user_df.to_csv(getcwd()+'/user_df.csv')
#reduced_triples.to_csv(getcwd()+'/reduced_triples_df.csv')
#song_info.to_csv(getcwd()+'/song_info_df.csv')

In [82]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(user_df.head(5))

                                          num_listens  listening_duration  \
user_id                                                                     
00729b0737db58cbd2d20b1037b3591beead6d72          169                 363   
00a9055cd54e8364080768065e13581544b7ff94          329                 191   
013de579fc16527ecaf104cb3a210f8258c99e05          103                 307   
021a670f55f7a440722f8508fc7629e4ff709ff3          268                 269   
032407039784e7302672e0ef0203f5836b518eca          229                 983   

                                                                                   song_map  \
user_id                                                                                       
00729b0737db58cbd2d20b1037b3591beead6d72                        {'TRBDQUF128F42AE54C': 169}   
00a9055cd54e8364080768065e13581544b7ff94                        {'TRAATBN128F14ACE82': 329}   
013de579fc16527ecaf104cb3a210f8258c99e05  {'TRAURYF128F147805D': 39, 'TRAWLYS129